# Project Group - 

Members & Student numbers: Willemijn ten Voorden (6101275), Frederiek Backers (4704452), Annerieke Ohm (4852990), Nikolaos Tsironis (6008712)

# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

This research proposal is made for the TIL Programming TIL6022 course project. The focus is on an application project on mobility trends using open-source data, making this a societal project. Inspiration was found in the current discussions on introducing rush hour tax for train tickets in The Netherlands (source: NS wil voorstel spitsheffing beperken tot €2,50 per rit
(https://www.businessinsider.nl/ns-wil-voorstel-spitsheffing-beperken-tot-e250-per-rit-veel-verzet-in-tweede-kamer/)).

This leads to the following research objective: “What are the main attributes of people choosing to travel by train in the Netherlands since 2018?”

Data processing, quantitative analysis and data visualisation will be used to answer this research question.

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**:

**Author 2**:

**Author 3**:

# Data Used

# Data Pipeline